In [1]:
from dask_jobqueue import SLURMCluster
from distributed import Client

In [2]:
cluster = SLURMCluster(queue='short', cores=4, memory='16gb', walltime='30:00', death_timeout=300)

In [3]:
client = Client(cluster)

In [5]:
client

Client Scheduler: tcp://10.120.16.104:54173 Dashboard: http://10.120.16.104:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [6]:
cluster.scale(n=100)

In [11]:
import tsfresh as tsf
import glob
import os
import numpy as np
import pandas as pd
import seaborn as sns

from importlib import reload
from scipy import signal
from tqdm.auto import tqdm
from matplotlib import pyplot as plt
from tsfresh.utilities.distribution import ClusterDaskDistributor
from src import main, feature_model, mp_profiler

seqs = []
samp_rate = '100ms'

# mp_profiler.init_yappi()
distributor = ClusterDaskDistributor(address=cluster.scheduler_address)

# for fp in tqdm(glob.glob('data/test_set/real-pd/testing_data/smartphone_accelerometer/*.csv')):
for fp in tqdm(glob.glob('data/test_set/cis-pd/testing_data/*.csv')):
    samp_id = os.path.splitext(os.path.basename(fp))[0]
    seq = main.read_seq(fp, use_time_index=True, resample=samp_rate, t_colname='Timestamp', xyz_colnames=['X', 'Y', 'Z'])
#     seq = main.read_seq(fp, use_time_index=True, resample=samp_rate)
    rms = np.sqrt(np.sum(np.square(seq), axis=1))
    df = pd.DataFrame(data={'rms': rms})
    df['id'] = samp_id
    seqs.append(df)

df = pd.concat(seqs)
print(f'resampled at {samp_rate}, sequences have {df.id.value_counts().mean()} samples on average')
results = tsf.extract_features(df, column_id='id', distributor=distributor)
results.to_csv('tsfeatures_cis_test.csv')


resampled at 100ms, sequences have 11620.459546925566 samples on average


In [13]:
cluster.scale(n=0)